# Fairness part of the Workshop
Analyze fairness of a dataset with different techniques

In [7]:
# Imports
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Loading data

We are first going to work with data from the Lending Club Dataset, a dataset of around 800k lending club users. This dataset doesn't have information about race or gender, so we will artificially create a "race" column to test our fairness metrics. Let's create a heavily unfair dataset: 80% of the users that were given a loan will be of race "1", and only 10% of the users who were not given a loan will be of race "1".

In [25]:
# Data management
df = pd.read_csv('../data/loans_data.csv')

# Subset
df = df.sample(frac=0.1)
display(df)

# Random vector to add noise to protected class
p_nochange=0.9
random_vec = np.random.choice([0,1], size=len(df), p=[p_nochange,1-p_nochange])

print(len(df), sum(random_vec))

# Add protected class
df['protected_class'] =  df['loan_status']^random_vec

print(df[['protected_class','loan_status']])


,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,fico_range_low,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,...,emp_length,home_ownership,verification_status,zip_code,term,initial_list_status,disbursement_method,application_type,loan_status,age_of_cr_line
433325,15000.0,0.0532,450000.00,3.26,0.0,720.0,0.0,34.446551,70.816146,20.0,...,< 1 year,MORTGAGE,Source Verified,973xx,36,w,Cash,Individual,1,20.0
623347,22375.0,0.2299,56000.00,20.02,0.0,680.0,0.0,34.446551,70.816146,13.0,...,10+ years,MORTGAGE,Source Verified,088xx,60,f,Cash,Individual,1,14.0
187502,20000.0,0.1972,71989.69,15.55,0.0,665.0,0.0,34.446551,96.000000,12.0,...,4 years,MORTGAGE,Verified,922xx,36,f,Cash,Individual,1,23.0
606239,15000.0,0.1199,73347.00,15.87,0.0,800.0,0.0,34.446551,70.816146,7.0,...,10+ years,MORTGAGE,Source Verified,799xx,60,w,Cash,Individual,1,24.0
104643,10000.0,0.0890,40000.00,20.82,2.0,715.0,0.0,7.000000,113.000000,10.0,...,10+ years,MORTGAGE,Source Verified,306xx,36,f,Cash,Individual,1,23.0
141698,8000.0,0.1114,75000.00,14.13,0.0,705.0,3.0,34.446551,70.816146,15.0,...,7 years,MORTGAGE,Source Verified,287xx,36,w,Cash,Individual,1,37.0
506217,25000.0,0.1465,97850.00,34.66,0.0,715.0,0.0,34.446551,70.816146,20.0,...,9 years,MORTGAGE,Not Verified,201xx,60,w,Cash,Individual,0,21.0
32809,10000.0,0.0894,65000.00,15.05,0.0,740.0,1.0,51.000000,70.816146,12.0,...,< 1 year,MORTGAGE,Not Verified,299xx,36,f,Cash,Individual,1,28.0
281119,25000.0,0.0769,70000.00,27.05,0.0,740.0,0.0,34.446551,70.816146,8.0,...,9 years,MORTGAGE,Source Verified,796xx,36,w,Cash,Individual,1,25.0
702279,15000.0,0.0532,40000.00,51.97,0.0,785.0,1.0,34.446551,70.816146,25.0,...,10+ years,MORTGAGE,Source Verified,582xx,36,w,Cash,Joint App,1,15.0


80515 8093
        protected_class  loan_status
433325                1            1
623347                1            1
187502                0            1
606239                1            1
104643                0            1
141698                1            1
506217                0            0
32809                 1            1
281119                1            1
702279                1            1
397124                1            1
89033                 1            1
22299                 0            1
389508                1            1
460022                0            1
782193                1            1
332723                1            1
267739                1            1
604083                1            1
146487                1            1
298882                0            1
36505                 1            1
366908                0            0
488015                1            1
648729                1            1
289918                1    

In [34]:
quant_cols = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low',\
              'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec',\
              'revol_bal', 'revol_util', 'total_acc', 'acc_now_delinq', 'tot_coll_amt',\
              'tot_cur_bal', 'tax_liens', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit',\
              'age_of_cr_line', 'installment','protected_class']

cat_cols = ['grade','emp_length', 'home_ownership','verification_status', 'term', 'initial_list_status',\
            'disbursement_method', 'application_type']

other_cols = ['zip_code']
response_col = 'loan_status'


# Get train and test
df_x = pd.get_dummies(df[quant_cols+cat_cols], drop_first=False, columns=cat_cols)
df_y = df[response_col]

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.3, random_state=42)

## Fitting our model

Let's fit a Random Forest to our data. Because we artificially added a biased "protected class" column, our classifier will not be fair.

In [35]:
# Fit model

model = RandomForestClassifier(n_estimators=25, max_depth=None, #class_weight='balanced_subsample', \
                               random_state=42).fit(x_train, y_train)

## Getting predictions

In [36]:
# Get predictions

preds_test = model.predict(x_test)

acc_train = model.score(x_train, y_train)
acc_test = model.score(x_test, y_test)

print(acc_train)
print(acc_test)

0.9988821859474805
0.9003104947215897


# Statistical Parity

We will first test our model's predictions with statistical parity, a simple fairness measure that is easy to compute.

## What is statistical parity?

This metric measures the difference between the probability of positive decisions for the protected group and the probability of positive decisions for ghe unprotected group. Mathematically:
$$Sp = P(d=1|G=0) - P(d=1|G=1)$$

This can be easily approximated with our data by calculating the proportion of positive decisions amongst people from race "0" and substracting the proportion of positive decisions amongst people from race "1":

$$Sp = \frac{ \text{# people with positive decision and race 0}} { \text{ # people from race 0} } - \frac{ \text{# people with positive decision and race 1}} { \text{ # people from race 1}}$$

Let's code a simple function that will calculate this for our dataset. In the next cell, complete the function `evaluate_statistical_parity` to perform the calculation above. The function definition and docstring will guide you.

In [117]:
# Statistical parity function
def evaluate_statistical_parity(predictions, protected_class_array):
        """Function to calculate statistical parity.
        
         Parameters
        ----------
        predictions (numpy array): binary decision labels outputted by our trained model.
        protected_class_array (numpy array): boolean mask where protected rows are marked True
        
        Returns
        -------
        bias (float): statistical parity bias
        """
        
        # Your code here
        
        prob_g = np.sum(predictions & protected_class_array) / np.sum(protected_class_array)
#         print(protected)
#         print( np.sum(h & protected),np.sum(protected))
        prob_not_g = np.sum(predictions & ~protected_class_array) / np.sum(~protected_class_array)
        bias = np.abs(prob_g - prob_not_g)
        return bias, prob_g, prob_not_g

# Conditional Parity

Statistical parity is a simple measure, and it gives a fast overview on our model's fairness. However, it disregards important aspects of our dataset, such as the values of the features of each row. We could have a situation where the statistical parity measure tells us that we are giving loans to 20% of people from race 0 and 20% of people from race 1, which would be fair, but those 20% from race 0 are random, while the 20% from race 1 are people from developed countries. Our model would be hiding another layer of unfairness: we are not giving loans equally to people from race 1.

We can use conditional parity to detect these types of imbalances. Conditional parity allows us to test for unfairness in a similar way as Statistical Parity, but conditioning on another feature (for example, country of origin). The equation is:

$$Cp = P(d=1|G=0, L=l) - P(d=1|G=1, L=l)$$

Again, this can be easily calculated by counting the number of positive outcome cases in por both protected groups, but this time only looking at the people that fulfill our conditional constraint (L=l)

In [118]:
# Conditional parity function
def evaluate_conditional_parity(predictions, protected_class_array, condition_array):
        """Function to calculate Conditional statistical parity.
        
         Parameters
        ----------
        predictions (numpy array): binary (decision) labels for X
        protected_class_array (numpy array): boolean array where protected rows are marked True
        condition_array (numpy array): boolean array that indicates conditional status
        
        Returns
        -------
        bias (float): conditional parity bias
        """
        
        # Your code here
        
        prob_g = np.sum(predictions & condition_array & protected_class_array) / np.sum(predictions & protected_class_array)
        prob_not_g = np.sum(predictions & condition_array & ~protected_class_array) / np.sum(predictions & ~protected_class_array)
        bias = np.abs(prob_g - prob_not_g)
        return bias, prob_g, prob_not_g

In [119]:
# Evaluate statistical and conditional parity
stat_parity = evaluate_statistical_parity([bool(x) for x in preds_test], ~x_test['protected_class'].apply(lambda x: bool(x)))
cond_parity = evaluate_conditional_parity([bool(x) for x in preds_test], ~x_test['protected_class'].apply(lambda x: bool(x)), x_test['loan_amnt']>10000)
print(stat_parity)
print(cond_parity)

(0.7682171773297348, 0.23178282267026512, 1.0)
(0.0465754959002308, 0.5397260273972603, 0.5863015232974911)


# FPR and FNR

The previous measures don't take into account the real labels of each observation; they only consider the predictions. The measure of fairness proposed here controls for equal poportions of false positives/false negatives in protected and unprotected classes. This measure is ideal in cases where committing mistakes disproportionately for different protected groups can bring negative outcomes.

We will again code these measures as they are rather easy to understand. The function definition below can help

In [80]:
# False positive and false negative rates
def evaluate_false_negative_rate(X, h, protected, y):
    """evaluate fnr

    Parameters
    ----------
    X (numpy array): input m x n matrix
    h (numpy array): binary (decision) labels for X
    protected (numpy array): boolean mask where protected rows are marked True
    y (numpy array): boolean array that marks ground truth

    Note:
        FNR: FN / CP where FN=(h==0) & (y==1) CN = (y==1)

    Returns
    -------
    bias (float)
    """

    cond_pos_protected = np.sum((y==1) & protected)
    cond_pos_not_protected = np.sum((y==1) & ~protected)
    if cond_pos_protected == 0:
        return 'No Condition Positive in Protected'
    if cond_pos_not_protected == 0:
        return 'No Condition Positive in Not Protected'

    false_neg_protected = np.sum((y==1) & (h==0) & protected)
    false_neg_not_protected = np.sum((y==1) & (h==0) & ~protected)

    fnr_g = false_neg_protected / cond_pos_protected
    fnr_not_g = false_neg_not_protected / cond_pos_not_protected
    bias = np.abs(fnr_g - fnr_not_g)
    return bias

def evaluate_false_positive_rate(X, h, protected, y):
        """evaluate fpr

        Parameters
        ----------
        X (numpy array): input m x n matrix
        h (numpy array): binary (decision) labels for X
        protected (numpy array): boolean mask where protected rows are marked True
        y (numpy array): boolean array that marks ground truth

        Note:
            FPR: FP / CN where FP=(h==1) & (y==0) CN = (y==0)

        Returns
        -------
        bias (float)
        """

        cond_neg_protected = np.sum((y==0) & protected)
        cond_neg_not_protected = np.sum((y==0) & ~protected)
        if cond_neg_protected == 0:
            return 'No Condition Negative in Protected'
        if cond_neg_not_protected == 0:
            return 'No Condition Negative in Not Protected'

        false_pos_protected = np.sum((y==0) & h & protected)
        false_pos_not_protected = np.sum((y==0) & h & ~protected)

        fpr_g = false_pos_protected / cond_neg_protected
        fpr_not_g = false_pos_not_protected / cond_neg_not_protected
        bias = np.abs(fpr_g - fpr_not_g)
        return bias


In [84]:
# Test FPR and FNR on this dataset

fnr = evaluate_false_negative_rate(x_test, preds_test, ~x_test['protected_class'], y_test)
fpr = evaluate_false_positive_rate(x_test, preds_test, ~x_test['protected_class'], y_test)

print(fpr)
print(fnr)

0.8325326012354153
0.6224066390041494


As we can see, the values of FPR and FNR are significantly higher than expected, showing that our dataset is clearly unfair.

## Other Fairness metrics 

We have coded and tested some basic Fairness metrics, but there are multiple other metrics that can be used, depending on the situation. Some of them are:

- Equalized odds
$$P(d=1|Y=i, G=m) = P(d=1|Y=i, G=m)$$
- Overall accuracy equality
- Treatment Equality
- Fairness through unawareness
- Fairness through awareness
- Disparate impact
Exists when decision outcomes disproportionately benefits or hurts individuals of a certain group.
- Disparate treatment
Decision changes when protected feature changes
- Disparate mistreatment:
Missclassification rates are different for people of different protected groups
- Preferential Fairness

We refer the reader to http://fairware.cs.umass.edu/papers/Verma.pdf for more information.

In [ ]:
# # Using FairML
# import audit_model

# #  call audit model with model
# total, _ = audit_model(clf.predict, propublica_data)

# # print feature importance
# print(total)

# # generate feature dependence plot
# fig = plot_dependencies(
#     total.get_compress_dictionary_into_key_median(),
#     reverse_values=False,
#     title="FairML feature dependence"
# )
# # plt.savefig("fairml_ldp.eps", transparent=False, bbox_inches='tight')

# Creating a Fair Model

Once we have characterized and measured the fairness of the model, we might want to build a model that avoids being unfair given a protected class. As there are multiple ways to define fairness, there are also multiple ways to build a fair classifier, depending on what notion we want to emphasize.

Some options are:
- Preprocessing the data to remove biases, and training normal classifiers on that data
- Training the classifier and post-processing the predictions to accomodate our measures of fairness
- Training a modified classifier that takes into account the protected class to avoid prediction imbalance


# Conclusion

We have analyzed particular fairness metrics and observed their behavior on an artificial dataset. It is important to remember that Fairness has multiple definitions, each one approriate for analyzing a specific situation. Statistical notions of fairness as described above are easy to measure. However, it is important to keep in mind that statistical definitions are insufficient in some cases (for example, when similarity has to be taken into account). Moreover, most valuable statistical metrics assume availability of actual, verified outcomes. While such outcomes are available for the training data, it is unclear whether the real classified data always conforms to the same distribution.

# Appendix: extra resources

## Interesting Fairness analysis tools
- audit-ai (https://github.com/pymetrics/audit-ai)
- fairness metrics (https://github.com/megantosh/fairness_measures_code)
- fairness-comparison (https://github.com/algofairness/fairness-comparison)
- IBM AIF360 (https://github.com/IBM/AIF360, https://arxiv.org/pdf/1810.01943.pdf)
- Themis ML (https://themis-ml.readthedocs.io/en/latest/)
- FairML (https://github.com/adebayoj/fairml)

## Interesting papers
- http://papers.nips.cc/paper/6988-optimized-pre-processing-for-discrimination-prevention.pdf (available in IBM AI360)
- http://fairware.cs.umass.edu/papers/Verma.pdf
- http://www.fatml.org/media/documents/from_parity_to_preference_notions_of_fairness.pdf
- http://papers.nips.cc/paper/6988-optimized-pre-processing-for-discrimination-prevention.pdf (by our very own Flavio Calmon)
- https://dl.acm.org/citation.cfm?doid=2783258.2783311


In [ ]:
# add IBM AIF360 examples if time